In [1]:
p_config = { # Trigger는 다시 당겨지고..
    "asic_mode":                      {"trigger": 1,  "value": 0},   # 0 training_only, 1 train_inf_sweep, 2 inference_only
    "training_epochs":                {"trigger": 2,  "value": 200},
    "inference_epochs":               {"trigger": 3,  "value": 500},
    "dataset":                        {"trigger": 4,  "value": 1},   # 0 DVS_GESTURE, 1 N_MNIST, 2 NTIDIGITS
    "timesteps":                      {"trigger": 5,  "value": 5},
    "input_size_layer1":              {"trigger": 6,  "value": 578},
    "long_time_input_streaming_mode": {"trigger": 7,  "value": 0},
    "binary_classifier_mode":         {"trigger": 8,  "value": 0},
    "loser_encourage_mode":           {"trigger": 9,  "value": 0},
    "layer1_cut_list":                {"trigger": 10, "value": [256,17,171,342,342,342,342,342,342,342,342,342,342,342,496]},
    "layer2_cut_list":                {"trigger": 11, "value": [256,17,171,342,342,342,342,342,342,342,342,342,342,342,496]},
}

import time
import numpy as np
from mms_ok import XEM7310

def signed_encoding_32bit(value):
    return value & 0xFFFFFFFF
def signed_decoding_32bit(value):
    return value - 0x100000000 if value & 0x80000000 else value

bitstream_path = r"bitstream/top_bh_fpga.bit"
fpga = XEM7310(bitstream_path=bitstream_path)


2026-01-02 22:30:46.807 | INFO     | mms_ok.ok_setup:copy_frontpanel_files:28 - FrontPanel SDK Version: 5.3.6
2026-01-02 22:30:46.810 | INFO     | mms_ok.ok_setup:copy_frontpanel_files:39 - FrontPanel API ready
2026-01-02 22:30:46.967 | INFO     | mms_ok.fpga:_validate_bitstream_path:117 - Bitstream file: c:\Users\User\measurement_setting\bitstream\top_bh_fpga.bit
2026-01-02 22:30:46.967 | INFO     | mms_ok.fpga:_validate_bitstream_path:120 - Bitstream date: 2026-01-02 22:25:50
2026-01-02 22:30:46.973 | INFO     | mms_ok.fpga:_connect:147 - Model        : XEM7310-A75
2026-01-02 22:30:46.973 | INFO     | mms_ok.fpga:_connect:148 - Serial Number: 1917000Q6P
2026-01-02 22:30:46.974 | INFO     | mms_ok.fpga:_connect:149 - Interface    : USB 3
2026-01-02 22:30:46.974 | INFO     | mms_ok.fpga:_connect:150 - USB Speed    : SUPER
2026-01-02 22:30:46.974 | INFO     | mms_ok.fpga:_connect:151 - Max Blocksize: 16384
2026-01-02 22:30:46.974 | INFO     | mms_ok.fpga:_connect:152 - Wire Width   : 32

In [2]:
# ######### Reset ######################################################################
reset_time = 1.0  # seconds
print(f'Resetting FPGA for {reset_time} seconds...')
fpga.reset(reset_address = 0x00, reset_time = reset_time, active_low = True)
print('System reset complete.')
time.sleep(1.0)
# ######### Reset ######################################################################

print('')
print('')
print('')
print('')

# fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
# ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# print(f'WireOut 0x20 value: {ep20wireout}')
# ep21wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x21))
# print(f'WireOut 0x21 value: {ep21wireout}')

# ######### p_config mode ######################################################################
config_ongoing_sleep_time = 0.01  # seconds
# config mode on
print(f'######### p_config mode ######################################################################')
fpga.SetWireInValue(0x01, signed_encoding_32bit(1))
fpga.ActivateTriggerIn(0x40, 0)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# asic_mode: 0 training_only, 1 train_inf_sweep, 2 inference_only 
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['asic_mode']['value']))
fpga.ActivateTriggerIn(0x40, p_config['asic_mode']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['asic_mode']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'WireOut 0x20 value: {ep20wireout} <-- asic_mode')
time.sleep(config_ongoing_sleep_time)
# training_epochs
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['training_epochs']['value']))
fpga.ActivateTriggerIn(0x40, p_config['training_epochs']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['training_epochs']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'WireOut 0x20 value: {ep20wireout} <-- training_epochs')
time.sleep(config_ongoing_sleep_time)
# inference_epochs
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['inference_epochs']['value']))
fpga.ActivateTriggerIn(0x40, p_config['inference_epochs']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['inference_epochs']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'WireOut 0x20 value: {ep20wireout} <-- inference_epochs')
time.sleep(config_ongoing_sleep_time)
# dataset: 0 DVS_GESTURE, 1 N_MNIST, 2 NTIDIGITS
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['dataset']['value']))
fpga.ActivateTriggerIn(0x40, p_config['dataset']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['dataset']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'WireOut 0x20 value: {ep20wireout} <-- dataset')
time.sleep(config_ongoing_sleep_time)
# timesteps
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['timesteps']['value']))
fpga.ActivateTriggerIn(0x40, p_config['timesteps']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['timesteps']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'WireOut 0x20 value: {ep20wireout} <-- timesteps')
time.sleep(config_ongoing_sleep_time)
# input_size_layer1_define
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['input_size_layer1']['value']))
fpga.ActivateTriggerIn(0x40, p_config['input_size_layer1']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['input_size_layer1']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'WireOut 0x20 value: {ep20wireout} <-- input_size_layer1')
time.sleep(config_ongoing_sleep_time)
# long_time_input_streaming_mode
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['long_time_input_streaming_mode']['value']))
fpga.ActivateTriggerIn(0x40, p_config['long_time_input_streaming_mode']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['long_time_input_streaming_mode']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'WireOut 0x20 value: {ep20wireout} <-- long_time_input_streaming_mode')
time.sleep(config_ongoing_sleep_time)
# binary_classifier_mode
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['binary_classifier_mode']['value']))
fpga.ActivateTriggerIn(0x40, p_config['binary_classifier_mode']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['binary_classifier_mode']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'WireOut 0x20 value: {ep20wireout} <-- binary_classifier_mode')
time.sleep(config_ongoing_sleep_time)
# loser_encourage_mode
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['loser_encourage_mode']['value']))
fpga.ActivateTriggerIn(0x40, p_config['loser_encourage_mode']['trigger'])
fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['loser_encourage_mode']['trigger']))
ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
print(f'WireOut 0x20 value: {ep20wireout} <-- loser_encourage_mode')
time.sleep(config_ongoing_sleep_time)
# layer1_cut_list
for i in range(15):
    fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['layer1_cut_list']['value'][i]))
    fpga.ActivateTriggerIn(0x40, p_config['layer1_cut_list']['trigger'])
    fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['layer1_cut_list']['trigger']))
    time.sleep(config_ongoing_sleep_time)
    ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
    print(f'WireOut 0x20 value: {ep20wireout} <-- layer1_cut_list[{i}]')
# layer2_cut_list
for i in range(15):
    fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['layer2_cut_list']['value'][i]))
    fpga.ActivateTriggerIn(0x40, p_config['layer2_cut_list']['trigger'])
    fpga.SetWireInValue(0x01, signed_encoding_32bit(p_config['layer2_cut_list']['trigger']))
    time.sleep(config_ongoing_sleep_time)
    ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
    print(f'WireOut 0x20 value: {ep20wireout} <-- layer2_cut_list[{i}]')
# ######### p_config mode######################################################################
    
    
print('')
print('')
print('')
print('')

# ######### p_config done mode ######################################################################
print(f'######### p_config done mode ######################################################################')
fpga.SetWireInValue(0x01, signed_encoding_32bit(2))
fpga.ActivateTriggerIn(0x40, 0)
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))
# ######### p_config done mode ######################################################################


    
print('')
print('')
print('')
print('')

# ep20wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x20))
# print(f'WireOut 0x20 value: {ep20wireout}')
# ep21wireout = signed_decoding_32bit(fpga.GetWireOutValue(0x21))
# print(f'WireOut 0x21 value: {ep21wireout}')


2026-01-02 22:30:47.400 | INFO     | mms_ok.fpga:reset:264 - Reset Start (1.0s)


Resetting FPGA for 1.0 seconds...


2026-01-02 22:30:48.415 | INFO     | mms_ok.fpga:reset:272 - Reset End (1.0s)



System reset complete.




######### p_config mode ######################################################################
WireOut 0x20 value: 0 <-- asic_mode
WireOut 0x20 value: 200 <-- training_epochs
WireOut 0x20 value: 500 <-- inference_epochs
WireOut 0x20 value: 1 <-- dataset
WireOut 0x20 value: 5 <-- timesteps
WireOut 0x20 value: 578 <-- input_size_layer1
WireOut 0x20 value: 0 <-- long_time_input_streaming_mode
WireOut 0x20 value: 0 <-- binary_classifier_mode
WireOut 0x20 value: 0 <-- loser_encourage_mode
WireOut 0x20 value: 256 <-- layer1_cut_list[0]
WireOut 0x20 value: 17 <-- layer1_cut_list[1]
WireOut 0x20 value: 171 <-- layer1_cut_list[2]
WireOut 0x20 value: 342 <-- layer1_cut_list[3]
WireOut 0x20 value: 342 <-- layer1_cut_list[4]
WireOut 0x20 value: 342 <-- layer1_cut_list[5]
WireOut 0x20 value: 342 <-- layer1_cut_list[6]
WireOut 0x20 value: 342 <-- layer1_cut_list[7]
WireOut 0x20 value: 342 <-- layer1_cut_list[8]
WireOut 0x20 value: 342 <-- layer1_cut_list[9]
WireOut 0x20 va

In [8]:
fpga.SetWireInValue(0x01, signed_encoding_32bit(0))


0

In [3]:
# fpga.close() 